<a href="https://colab.research.google.com/github/Grace1903/mentalhealthChatbot/blob/main/Mentalhealthchatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install langchain_groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 4.6 MB/s eta 0:00:00


In [4]:
from langchain_groq import ChatGroq
llm= ChatGroq(
    temperature=0,
    groq_api_key ="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",
    model_name ="llama-3.3-70b-versatile"
)
result = llm.invoke("Who are you?")
print(result.content)

I'm an artificial intelligence model known as Llama. Llama stands for "Large Language Model Meta AI."


In [5]:
!pip install pypdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.6/304.6 kB 5.9 MB/s eta 0:00:00


In [11]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader # Changed from PyPDFDirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq # Import ChatGroq
import os # Import os

def initialize_llm():
  llm = ChatGroq(
      temperature=0,
    groq_api_key ="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",
    model_name ="llama-3.3-70b-versatile"

  )
  return llm

def create_vector_db():
  # loader = DirectoryLoader("/content/data/", glob ='*.pdf', loader_cls = PyPDFDirectoryLoader) # Commented out DirectoryLoader
  loader = PyPDFLoader("/content/data/mental_health_Document_250625_161048.pdf") # Use PyPDFLoader with specific file path
  documents = loader.load()
  print(f"Number of documents loaded: {len(documents)}") # Added print statement
  text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500 , chunk_overlap = 50)
  texts = text_splitter.split_documents(documents)
  embeddings = HuggingFaceEmbeddings(model_name ='sentence-transformers/all-MiniLM-L6-v2') # Corrected typo
  vector_db = Chroma.from_documents(texts,embeddings, persist_directory='./chroma_db') # Corrected variable name
  vector_db.persist()

  print("ChromaDB created and data saved")
  return vector_db

def setup_qa_chain(vector_db, llm):
  retriever=vector_db.as_retriever()
  prompt_template =""" You are a compassionate mental health chatbot.Respond thoughtfully to the following question:\n",
  "{context}\n",
  "User:{question}\n",
  " Chatbot:"""
  PROMPT =PromptTemplate(template = prompt_template, input_variables =['context','question'])
  qa_chain =RetrievalQA.from_chain_type(
      llm = llm,
      chain_type ="stuff",
      retriever = retriever ,
      chain_type_kwargs = {"prompt":PROMPT}

  )
  return qa_chain

def main():
  print("initializing Chatbot")
  llm = initialize_llm()

  db_path ="./chroma_db" # Corrected path
  if os.path.exists(db_path): # Added check for db_path existence
    print("Loading existing ChromaDB")
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2') # Corrected typo
    vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)
  else:
    print("Creating new ChromaDB")
    vector_db = create_vector_db()

  qa_chain= setup_qa_chain(vector_db,llm)

  while True:
    query =input("\nHuman:")
    if query.lower() == "exit":
      print("Chatbot:take care of yourself,Goodbye")
      break
    response = qa_chain.run(query)
    print(f"Chatbot:{response}")


if __name__ == "__main__":
  main()

initializing Chatbot
Creating new ChromaDB
Number of documents loaded: 14


/tmp/ipython-input-11-1053724064.py:26: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name ='sentence-transformers/all-MiniLM-L6-v2') # Corrected typo
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to ac

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/tmp/ipython-input-11-1053724064.py:28: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_db.persist()


ChromaDB created and data saved

Human:hello


/tmp/ipython-input-11-1053724064.py:69: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain.run(query)


Chatbot:Hello! It's nice to meet you. I see you've been reading about mental health and the various definitions surrounding it. That's a great topic to explore. The concept of mental health can be complex and multifaceted, and it's interesting to see how different researchers and organizations define it.

From what I've gathered, it seems like you're looking at definitions that emphasize the importance of having a sense of purpose, quality relationships, self-respect, and mastery. These are all important aspects of mental well-being, and it's great that you're thinking about what contributes to a good life.

I'd like to add that mental health is indeed influenced by our social and physical environments, as you mentioned. It's not just about individual factors, but also about the world around us and how we interact with it. This is why it's so important to consider the broader context when thinking about mental health.

What are your thoughts on this topic? What do you think is most imp

In [12]:
!pip install gradio


In [13]:
create_vector_db()

Number of documents loaded: 14
ChromaDB created and data saved


In [9]:
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [8]:
!pip install langchain_groq langchain_core langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.6 MB/s eta 0:00:00


In [7]:
!pip install Chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 78.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 74.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 6.2 MB/s eta 

In [ ]:
!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [ ]:
import os

print(os.listdir("/content/data/"))

['mental_health_Document_250625_161048.pdf']


In [ ]:
create_vector_db()
main()

Number of documents loaded: 14
ChromaDB created and data saved


NameError: name 'main' is not defined

In [ ]:
import os
print(os.listdir("/content/data/"))

['mental_health_Document_250625_161048.pdf']


In [1]:
!mkdir data

In [2]:
import os
print(os.listdir("/content/data/"))

['mental_health_Document_250625_161048.pdf']


In [26]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader # Changed from PyPDFDirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq # Import ChatGroq
import os # Import os
import gradio as gr

def initialize_llm():
  llm = ChatGroq(
      temperature=0,
    groq_api_key ="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",
    model_name ="llama-3.3-70b-versatile"

  )
  return llm

def create_vector_db():
  # loader = DirectoryLoader("/content/data/", glob ='*.pdf', loader_cls = PyPDFDirectoryLoader) # Commented out DirectoryLoader
  loader = PyPDFLoader("/content/data/mental_health_Document_250625_161048.pdf") # Use PyPDFLoader with specific file path
  documents = loader.load()
  print(f"Number of documents loaded: {len(documents)}") # Added print statement
  text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500 , chunk_overlap = 50)
  texts = text_splitter.split_documents(documents)
  embeddings = HuggingFaceEmbeddings(model_name ='sentence-transformers/all-MiniLM-L6-v2') # Corrected typo
  vector_db = Chroma.from_documents(texts,embeddings, persist_directory='./chroma_db') # Corrected variable name
  vector_db.persist()

  print("ChromaDB created and data saved")
  return vector_db

def setup_qa_chain(vector_db, llm):
  retriever=vector_db.as_retriever()
  prompt_template =""" You are a compassionate mental health chatbot.Respond thoughtfully to the following question:\n",
  "{context}\n",
  "User:{question}\n",
  " Chatbot:"""
  PROMPT =PromptTemplate(template = prompt_template, input_variables =['context','question'])
  qa_chain =RetrievalQA.from_chain_type(
      llm = llm,
      chain_type ="stuff",
      retriever = retriever ,
      chain_type_kwargs = {"prompt":PROMPT}

  )
  return qa_chain

def chatbot_response(user_input, history): # Keep history parameter as Gradio passes it
  if not user_input.strip():
    return "please provide a valid input" # Return only the response string
  # Initialize LLM and QA chain within the response function or pass them
  llm = initialize_llm()
  db_path ="./chroma_db"
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
  vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)
  qa_chain= setup_qa_chain(vector_db,llm)

  response=qa_chain.run(user_input) # Call qa_chain.run with user_input
  return response # Return only the response string


if __name__ == "__main__":
  print("initializing Chatbot backend")
  llm = initialize_llm()

  db_path ="./chroma_db" # Corrected path
  if os.path.exists(db_path): # Added check for db_path existence
    print("Loading existing ChromaDB")
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2') # Corrected typo
    vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)
  else:
    print("Creating new ChromaDB")
    vector_db = create_vector_db()

  qa_chain= setup_qa_chain(vector_db,llm)

  with gr.Blocks(theme= 'Repair/SHIKI@1.2.1')as app:
       gr.Markdown("Mental Health Chatbot ") # Fixed syntax error
       gr.Markdown("A compassionate chatbot designed to assist with mental well-being. Please note: For serious concerns, contact a professional.") # Fixed syntax error

       chatbot = gr.ChatInterface(fn=chatbot_response, title="Mental Health Chatbot")

       gr.Markdown("This chatbot provides general support. For urgent issues, seek help from licensed professionals.")

    # Modified fn to correctly pass history and expect a tuple return
    # Removed duplicate ChatInterface line
  app.launch(debug=True) # Added debug=True for more detailed output

initializing Chatbot backend
Loading existing ChromaDB


/usr/local/lib/python3.11/dist-packages/gradio/blocks.py:1153: UserWarning: Cannot load Repair/SHIKI@1.2.1. Caught Exception: The space Repair/SHIKI does not exist
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")
/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:339: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://a7dc3cdd5992201bcf.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://7a64ac52089731396b.gradio.live
Killing tunnel 127.0.0.1:7860 <> https://a7dc3cdd5992201bcf.gradio.live


In [27]:
!pip install nbstripout
!nbstripout your_notebook.ipynb



Could not strip 'your_notebook.ipynb': file not found


In [28]:
from google.colab import files
uploaded = files.upload()


KeyboardInterrupt: 

In [29]:
{'my_notebook.ipynb': 'my_notebook.ipynb'}


{'my_notebook.ipynb': 'my_notebook.ipynb'}

In [30]:
!nbstripout my_notebook.ipynb


Could not strip 'my_notebook.ipynb': file not found
